In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import time

2024-01-13 15:13:14.683247: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-13 15:13:14.683319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-13 15:13:14.685211: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-13 15:13:14.695442: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-13 15:13:15.534148: W tensorflow/comp

In [73]:
f = np.load("mfcc_fixed.npz")
X, Y = f['X'], f['Y']
X_train =  X[0:900]
Y_train =  Y[0:900]        # fold 10


x_test, y_test = X[900:1000], Y[900:1000]
x_train, y_train = X_train[0:800], Y_train[0:800]
x_val, y_val= X_train[800:900], Y_train[800:900]

In [74]:
batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=800, reshuffle_each_iteration=True).batch(batch_size, drop_remainder=True)

In [75]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(),
                      metrics=['accuracy'])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.hdf5'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model




In [76]:
model = Classifier_INCEPTION("output", X[0].shape, 5)

In [77]:
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
lds = lambda x, y: tf.math.reduce_sum(keras.losses.kl_divergence(x, y))
acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [78]:
es_callback = keras.callbacks.EarlyStopping(
                                            monitor="val_loss",
                                            min_delta=0,
                                            patience=100,
                                            verbose=1,
                                            mode="auto",
                                            baseline=None,
                                            restore_best_weights=True,
                                            start_from_epoch=0)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                              min_lr=0.0001)
model.model.fit(train_dataset, validation_data=val_dataset, batch_size=batch_size, epochs=2000,
          callbacks=[es_callback, reduce_lr])

Epoch 1/2000
12/12 [==============================] - 8s 62ms/step - loss: 0.8725 - accuracy: 0.7409 - val_loss: 28.1642 - val_accuracy: 0.5156 - lr: 0.0010
Epoch 2/2000
12/12 [==============================] - 0s 18ms/step - loss: 0.1868 - accuracy: 0.9414 - val_loss: 27.5716 - val_accuracy: 0.4531 - lr: 0.0010
Epoch 3/2000
12/12 [==============================] - 0s 20ms/step - loss: 0.0911 - accuracy: 0.9779 - val_loss: 25.0769 - val_accuracy: 0.2188 - lr: 0.0010
Epoch 4/2000
12/12 [==============================] - 0s 17ms/step - loss: 0.0420 - accuracy: 0.9896 - val_loss: 11.7978 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 5/2000
12/12 [==============================] - 0s 16ms/step - loss: 0.0205 - accuracy: 0.9987 - val_loss: 12.2792 - val_accuracy: 0.3438 - lr: 0.0010
Epoch 6/2000
12/12 [==============================] - 0s 17ms/step - loss: 0.0119 - accuracy: 0.9987 - val_loss: 6.9422 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 7/2000
12/12 [==============================] - 0s 20

In [79]:
acc_metric.reset_states()
acc_metric.update_state(y_test, model.model(x_test))
acc = acc_metric.result().numpy()
loss = loss_fn(y_test, model.model(x_test)).numpy()
print(acc, loss)

1.0 1.8953191e-05


In [80]:
model.model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 54, 30)]             0         []                            
                                                                                                  
 conv1d_288 (Conv1D)         (None, 54, 32)               960       ['input_10[0][0]']            
                                                                                                  
 max_pooling1d_54 (MaxPooli  (None, 54, 30)               0         ['input_10[0][0]']            
 ng1D)                                                                                            
                                                                                                  
 conv1d_289 (Conv1D)         (None, 54, 32)               40960     ['conv1d_288[0][0]']    